In [1]:
import json
import datetime
from dataclasses import dataclass
import numpy as np
import pandas as pd

In [2]:
f = open('/Users/ianmckinley/Documents/9.1.json')

file = json.load(f)

f.close()

In [ ]:
@dataclass
class file_parser():

    full_data = pd.DataFrame(columns=['spk','start_time','end_time','transcript'])

    def __init__(self):
        self.spk:str = "spk_0"
        self.start_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
        self.end_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
        self.transcript:str = ""

    #We can potentially write the function to parse the file within the class
    #Haven't decided if this is the optimal way to do it or if it needs to be formatted differently
    #Will work on the parsing function itself and decide where it should live later


In [3]:
#Testing cell to show what gets printed with various access keys and indices

for item in file['results']['items']:
    print(item)
    print(item['alternatives'][0]['content'])
    print(item['start_time'])
    break

{'type': 'pronunciation', 'alternatives': [{'confidence': '0.99', 'content': 'Keep'}], 'start_time': '2.609', 'end_time': '2.96', 'speaker_label': 'spk_0'}
Keep
2.609


In [4]:
start_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
end_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)

full_data = pd.DataFrame(columns=['spk','start_time','end_time','transcript'])

second_start=int()
minute_start=int()
second_end=int()
minute_end=int()
spk = "spk_0"
transcript = ""

#This segment is the broadest loop, which is pulling each 'item' from the Amazon-produced JSON file and parsing it to the useful chunks we need
for item in file['results']['items']:
    
    #This block checks if the speaker label has changed at its outermost scope, and behaves differently depending on whether or not the speaker has changed
    #We may need to change the behavior at this level when we start using multi channel input in the future
    if item['speaker_label'] == spk:
        
        #First we have to see if the item is a punctuation mark, and if so, we need to add it to the transcript without a space and without changing the timestamp
        if item['type'] == 'punctuation':
            transcript = transcript + item['alternatives'][0]['content']
        elif item['type'] == 'pronunciation':

            #If the speaker hasn't changed, we add the content to the transcript
            #I plan to come back here later and add a checking function, where if the item type is 'punctuation' it will omit the added space before addending the content
            transcript = transcript + " " + item['alternatives'][0]['content']
        
            while True:
                try:
                    #We also check to see if the start time is listed as being 0.0 (i.e. if it has been reset) and if so, we set it to the first value we come across since the reset
                    #This allows us to set the correct start time for each line of text per speaker, which will be important for assigning pauses in the future
                    if start_time.total_seconds() == 0.0:
                        split_start = item['start_time'].split('.')
                        second_start = int(split_start[0])
                        millisecond_start = (split_start[1])
                        if len(millisecond_start) == 1:
                            millisecond_start = int(millisecond_start) * 100
                        elif len(millisecond_start) == 2:
                            millisecond_start = int(millisecond_start) * 10
                        else:
                            millisecond_start = int(millisecond_start)
                        start_time_hold = datetime.timedelta(days=0,seconds=second_start,milliseconds=millisecond_start)
                
                        #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
                        s1= start_time_hold.total_seconds()
                        hours1, remain1 = divmod(s1, 3600)
                        minutes1, remain2_1 = divmod(remain1, 60)
                        #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
                        seconds1, remain3_1 = divmod(remain2_1, 1)
                        #We added this modifier to increase the remainder so the milliseconds format correctly
                        remain3_1 = remain3_1 * 1000
                        var1 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hours1), int(minutes1), int(seconds1), int(remain3_1)))

                        start_time = var1
                        break
                    else:
                        break
                except AttributeError:
                    break
        
            #This block takes the end time from the JSON file and parses it into a timedelta object by splitting it and then putting it back together in the desired format
            split_end = item['end_time'].split('.')
            second_end = int(split_end[0])
            millisecond_end = (split_end[1])
            if len(millisecond_end) == 1:
                millisecond_end = int(millisecond_end) * 100
            elif len(millisecond_end) == 2:
                millisecond_end = int(millisecond_end) * 10
            else:
                millisecond_end = int(millisecond_end)
            end_time_hold = datetime.timedelta(days=0,seconds=second_end,milliseconds=millisecond_end)
            
            #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
            s2= end_time_hold.total_seconds()
            hours2, remain2 = divmod(s2, 3600)
            minutes2, remain2_2 = divmod(remain2, 60)
            #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
            seconds2, remain3_2 = divmod(remain2_2, 1)
            #We added this modifier to increase the remainder so the milliseconds format correctly
            remain3_2 = remain3_2 * 1000
            var2 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hours2), int(minutes2), int(seconds2), int(remain3_2)))
            
            #Then we replace the end time with the holder, to be sure it is updated every time and no failures will occur
            end_time = var2
    
    #This is the block which is called if the speaker changes
    elif item['speaker_label'] != spk:
        
        #We add the data currently in our containers to the dataframe
        full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'transcript':transcript},ignore_index=True)
        
        #Change the speaker label to the new speaker
        spk = item['speaker_label']
        
        #Reset the start time to the current value
        split_start = item['start_time'].split('.')
        second_start = int(split_start[0])
        millisecond_start = (split_start[1])
        if len(millisecond_start) == 1:
            millisecond_start = int(millisecond_start) * 100
        elif len(millisecond_start) == 2:
            millisecond_start = int(millisecond_start) * 10
        else:
            millisecond_start = int(millisecond_start)
        start_time_hold = datetime.timedelta(days=0,seconds=second_start,milliseconds=millisecond_start)
        
        #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
        s3= start_time_hold.total_seconds()
        hours3, remain3 = divmod(s3, 3600)
        minutes3, remain2_3 = divmod(remain3, 60)
        #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
        seconds3, remain3_3 = divmod(remain2_3, 1)
        #We added this modifier to increase the remainder so the milliseconds format correctly
        remain3_3 = remain3_3 * 1000
        var3 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hours3), int(minutes3), int(seconds3), int(remain3_3)))
        start_time = var3

        #This block takes the end time from the JSON file and parses it into a timedelta object by splitting it and then putting it back together in the desired format
        split_end = item['end_time'].split('.')
        second_end = int(split_end[0])
        millisecond_end = (split_end[1])
        if len(millisecond_end) == 1:
            millisecond_end = int(millisecond_end) * 100
        elif len(millisecond_end) == 2:
            millisecond_end = int(millisecond_end) * 10
        else:
            millisecond_end = int(millisecond_end)
        end_time_hold = datetime.timedelta(days=0,seconds=second_end,milliseconds=millisecond_end)
        #Lifted this directly from https://stackoverflow.com/a/539360 & modified it slightly for own use
        sx= end_time_hold.total_seconds()
        hoursx, remainx = divmod(sx, 3600)
        minutesx, remainx2 = divmod(remainx, 60)
        #Remember that divisor on the following line is 1, because the whole thing is indexed to seconds
        secondsx, remainx3 = divmod(remainx2, 1)
        #We added this modifier to increase the remainder so the milliseconds format correctly
        remainx3 = remainx3 * 1000
        var4 = ('{:02}:{:02}:{:02}:{:03}'.format(int(hoursx), int(minutesx), int(secondsx), int(remainx3)))
        end_time = var4

        #Reset the transcript to be blank
        transcript = ""
        #Then add the current content to the transcript
        transcript = transcript + " " + item['alternatives'][0]['content']

#When the whole thing has run, the final line of data will not have been added to the dataframe, so we add it here
full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'transcript':transcript},ignore_index=True)



/var/folders/cl/6jd7_6591xn2wyp1gzj9t8340000gn/T/ipykernel_49291/1422683644.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'transcript':transcript},ignore_index=True)
/var/folders/cl/6jd7_6591xn2wyp1gzj9t8340000gn/T/ipykernel_49291/1422683644.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'transcript':transcript},ignore_index=True)
/var/folders/cl/6jd7_6591xn2wyp1gzj9t8340000gn/T/ipykernel_49291/1422683644.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = full_data.append({'spk':spk,'start_time':start_time,'end_time':end_time,'trans

In [ ]:
full_data.to_csv('/Users/ianmckinley/Documents/9.1.csv_dataframe.csv',index=True)

In [ ]:
start_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)
end_time = datetime.timedelta(days=0,seconds=0,milliseconds=0)

full_data = pd.DataFrame(columns=['spk','start_time','end_time','transcript'])

In [5]:

full_data

,spk,start_time,end_time,transcript
0,spk_0,00:00:02:609,00:00:09:509,Keep all that fat up. Yeah. Look at the other...
1,spk_1,00:00:12:239,00:00:22:818,I need to be gone. But uh
2,spk_0,00:00:26:940,00:00:37:679,Right. Yeah.
3,spk_1,00:00:39:240,00:00:46:798,Did I get the camera cleans? The only thing I...
4,spk_0,00:00:57:118,00:01:19:829,Sorry we take it. Oh no. Mhm.
...,...,...,...,...
332,spk_0,02:43:19:540,02:43:21:440,And then don't get the stomach again
333,spk_1,02:43:21:489,02:43:26:009,back travel but not take stomach but a wide b...
334,spk_0,02:43:26:019,02:44:48:190,"like uh yeah, cause you see how you almost go..."
335,spk_1,02:44:51:030,02:44:52:099,one more and then cut
